In [1]:
from huggingface_hub import notebook_login

# Authenticate to Hugging Face in a notebook environment
notebook_login()

In [12]:
# Create CSV file that lists the content in /data
import os
import csv
import pandas as pd
import sys

data_dir = os.path.join(os.getcwd(), "data")
images_dir = os.path.join(data_dir, "images")
print(f"Images dir: {images_dir}")
repo_dir = os.path.join(data_dir, "repos")
compress_repo_dir = os.path.join(data_dir, "compressed_repos")
csv_file = os.path.join("dataset.csv")

os.makedirs(compress_repo_dir, exist_ok=True)

# The CSV should contain: id |image_path | compressed_repo_path | labels (void for now)
df = pd.DataFrame(columns=["id", "image", "compressed_repo", "labels"])
i = 0

for root, dirs, files in os.walk(images_dir):
    print(f"Root: {root}")
    print(f"Dirs: {dirs}")
    print(f"Files: {files}")
    for file in files:
        file_name_without_extension = os.path.splitext(file)[0]
        if not file.endswith(".png"):
            print(f"Skipping {file} as it is not a PNG file")
            continue

        # Check that there is a repo with the same name
        repo_file = os.path.join(repo_dir, file_name_without_extension)
        if not os.path.exists(repo_file):
            print(f"Skipping, as the repo {repo_file} does not exist")
            continue

        # Zip the repository (directory) and save it in /compressed_repo
        compress_repo_file = os.path.join(compress_repo_dir, file_name_without_extension + ".zip")
        os.system(f"zip -r {compress_repo_file} {repo_file}")

        # Write the row in the df
        df = pd.concat([df, pd.DataFrame([[i, os.path.join(images_dir, file), compress_repo_file, ""]], columns=df.columns)])
        i += 1
        
# Save the df as a CSV
df.to_csv(csv_file, index=False)

Images dir: /home/josselin/VLM-image2structure-eval/git-webpages-experiment/github-webpage-scraper/data/images
Root: /home/josselin/VLM-image2structure-eval/git-webpages-experiment/github-webpage-scraper/data/images
Dirs: []
Files: ['16_nirowu_github.io.png', '25_collinsonindo_weatherapp.github.io.png', '47_jumptask-io_jumptask-io.github.io.png', '0_paulabravic_paulabravic.github.io.png', '18_TatianaLouise_github.io.png', '64_vinoth-kannan_vinth-kannan-github.io1.png', '80_arpit898_agtgdate.github.io.png', '71_Ishanshi-B_gooutwebsite.github.io.png', '14_ryoryotarosu_github.io.png', '31_GuillermoJCV_GuillermoJCV.github.io.png', '38_IonutMitoi_ionutmitoi.github.io.png', '13_elektrikaam_github.io.png', '4_tuiteru_github.io.png', '29_imanawesome_imanawesome.github.io.png', '70_GunashriR_https-GunashriR.github1.io.png', '45_SanviGitHub_SanviGithub.github.io.png', '40_Yare33antone_Odksurvivor.github.io40.png', '50_ranakisi_github.io-.png', '57_PrestonNguyen23_preston-nguyen.github.io2.png', 

In [15]:
from PIL import Image
import io
from datasets import Dataset, DatasetDict
import pandas as pd
from sklearn.model_selection import train_test_split
from huggingface_hub import HfApi, HfFolder
from toolbox.printing import debug
import _random
import base64
CAT_INDEX = 3

df = pd.read_csv(f"dataset.csv")
# Shuffle dataframe
df = df.sample(frac=1).reset_index(drop=True)
train_df, valid_df = train_test_split(df, test_size=0.2)

def load_image(image_path):
    if image_path.lower().endswith(".png") or image_path.lower().endswith(".jpg"):
        with open(image_path, "rb") as f:
            return Image.open(io.BytesIO(f.read()))
    else:
        with open(image_path, "rb") as f:
            return io.BytesIO(f.read())

def read_file(file_path):
    with open(file_path, "r") as file:
        return file.read()
    
def read_compressed_repo(zip_file_path):
    with open(zip_file_path, 'rb') as zip_file:
        zip_content = zip_file.read()
    base64_encoded = base64.b64encode(zip_content).decode('utf-8')
    return base64_encoded

def transform(row):
    row["compressed_repo"] = read_compressed_repo(row["compressed_repo"])
    row["image"] = load_image(row["image"])
    return row

train_dataset = Dataset.from_pandas(train_df).map(transform).shuffle()
valid_dataset = Dataset.from_pandas(valid_df).map(transform).shuffle()

from datasets import Features, Value, Image, load_dataset

# # Define the features of your dataset
# features = Features({
#     'id': Value('int64'),
#     'image': Value('string'),
#     'category': Value('string'),
#     'subject': Value('string'),
#     'output': Image(),        # For image features
#     'asset_0': Value('string'),
#     'asset_1': Value('string'),
#     'asset_2': Value('string'),
#     'asset_3': Value('string'),
#     'asset_4': Value('string'),
#     'asset_5': Value('string'),
#     'asset_6': Value('string'),
#     'asset_7': Value('string'),
#     'asset_8': Value('string'),
#     'asset_9': Value('string'),
#     '__index_level_0__': Value('int64')
# })

# train_dataset = train_dataset.cast(features)
# valid_dataset = valid_dataset.cast(features)

dataset_dict = DatasetDict({"train": train_dataset, "validation": valid_dataset})
# help(dataset_dict.push_to_hub)
# Then save or upload as before


dataset_dict.push_to_hub("stanford-crfm/i2s-webpage")

Map:   0%|          | 0/70 [00:00<?, ? examples/s]

Map:   0%|          | 0/18 [00:00<?, ? examples/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/stanford-crfm/i2s-webpage/commit/ca7dd37ee9cdafa0a1e2cac33301d886e7a41a85', commit_message='Upload dataset', commit_description='', oid='ca7dd37ee9cdafa0a1e2cac33301d886e7a41a85', pr_url=None, pr_revision=None, pr_num=None)

In [19]:
path = "/home/josselin/VLM-image2structure-eval/git-webpages-experiment/github-webpage-scraper/data/compressed_repos/0_paulabravic_paulabravic.github.io.zip"
s = read_compressed_repo(path)

In [20]:
s

'...'

In [ ]:
import base64
import io
import zipfile

# Function to decode base64 string and extract zip content
def decode_and_unzip_base64(base64_string, extract_to_folder):
    zip_content = base64.b64decode(base64_string)
    with zipfile.ZipFile(io.BytesIO(zip_content), 'r') as zip_ref:
        zip_ref.extractall(extract_to_folder)

# Example usage
base64_string = s
extract_to_folder = 'unzipped_folder'
decode_and_unzip_base64(base64_string, extract_to_folder)